In [ ]:
# 이론 참고
https://machinelearningmastery.com/the-luong-attention-mechanism/

In [1]:
# encoder decoer 참고
https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/
https://lsjsj92.tistory.com/449
    https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

SyntaxError: invalid syntax (<ipython-input-1-cc2cd28cca59>, line 2)

In [ ]:
# attention 참고
https://betterprogramming.pub/a-guide-on-the-encoder-decoder-model-and-the-attention-mechanism-401c836e2cdb

In [67]:
# path = 'C:/Users/100sh/python/algorithm/데이터셋/kor_eng/'
path = 'C:/Users/100sh/python/algorithm/데이터셋/spa-eng/'
with open(path + 'spa.txt', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
lines

['Go.\tVe.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986655 (cueyayotl)',
 'Go.\tVete.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986656 (cueyayotl)',
 'Go.\tVaya.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986657 (cueyayotl)',
 'Go.\tVáyase.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #6586271 (arh)',
 'Hi.\tHola.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #431975 (Leono)',
 'Run!\t¡Corre!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1685404 (Elenitigormiti)',
 'Run!\t¡Corran!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5213896 (cueyayotl)',
 'Run!\t¡Corra!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #8005613 (Seael)',
 'Run!\t¡Corred!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #8005615 (Seael)',
 'Run.\tCorred.\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #6681472 (arh)',
 'Who?\

### 1. 데이터 전처리

In [68]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [69]:
# 한글, 영어 분리해주기
eng_data = []
kor_data = []
input_characters = set()
target_characters = set()

cnt = 1
for v in lines:
    v = v.split('\t')
    kor_sent = v[1]
    kor_sent = '\t'+ kor_sent + '\n'
    eng_sent = v[0]
    eng_data.append(eng_sent)
    kor_data.append(kor_sent)  # 시작점과 끝점을 알리기 위해 \n, \n 넣는다.
    for char in eng_sent:
        if char not in input_characters:
            input_characters.add(char)
    for char in kor_sent:
        if char not in target_characters:
            target_characters.add(char)        
    cnt += 1
    if cnt == len(lines):
        break
        
        
# 용량이 너무 커서
eng_data = eng_data[:1000]
kor_data = kor_data[:1000]


print(len(eng_data))
print(len(kor_data))
print(len(lines))  # lines 마지막 행에 데이터가 없음
print(eng_data[:5])
print(kor_data[:5])

print(random.sample(input_characters, 10))  # 영어단어안에 있는 모든 철자 데이터
print(random.sample(target_characters, 10)) # 한글 단어안에 있는 모든 철자 데이터

1000
1000
138441
['Go.', 'Go.', 'Go.', 'Go.', 'Hi.']
['\tVe.\n', '\tVete.\n', '\tVaya.\n', '\tVáyase.\n', '\tHola.\n']
['y', '5', '4', 'l', 'b', 'n', '!', 'R', 'G', ':']
['J', '°', 'p', 'º', 'R', 'ó', '\t', "'", 'Q', '(']


In [70]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(i) for i in eng_data])
max_decoder_seq_length = max([len(i) for i in kor_data])
print('number of unique input tokens:', num_encoder_tokens)
print('number of unique output tokens:',num_decoder_tokens )
print('input max length:', max_encoder_seq_length)
print('output max length:', max_decoder_seq_length)

number of unique input tokens: 92
number of unique output tokens: 112
input max length: 10
output max length: 31


In [71]:
target_characters[:10]

['\t', '\n', ' ', '!', '"', '$', '%', '&', "'", '(']

In [72]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

for key, value in input_token_index.items():
    if value <10 : print(key, ".", value)


  . 0
! . 1
" . 2
$ . 3
% . 4
' . 5
( . 6
) . 7
+ . 8
, . 9


In [73]:
# ont hot 데이터를 넣을 빈 데이터 만들기
encoder_input_data = np.zeros((len(eng_data), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(eng_data), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(eng_data), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

test_input_data = np.zeros((len(eng_data), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')

print(np.shape(encoder_input_data))
print(np.shape(decoder_input_data))
print(np.shape(decoder_target_data))
print(np.shape(test_input_data))

(1000, 10, 92)
(1000, 31, 112)
(1000, 31, 112)
(1000, 10, 92)


In [74]:
# one hot encoding test 
# 단어 있는 위치에만 1이 찍힘
for i, (eng, kor) in enumerate(zip(eng_data, kor_data)):
    for t, char in enumerate(eng):
        print(i, t, input_token_index[char])
        print(char)
        test_input_data[i, t, input_token_index[char]] = 1.

    break
test_input_data[0][0]

0 0 32
G
0 1 66
o
0 2 11
.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [75]:
# one hot encoding test 
# 단어 있는 위치에만 1이 찍힘
for i, (eng, kor) in enumerate(zip(eng_data, kor_data)):
    for t, char in enumerate(eng):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(kor):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0: # target data는 tiime step 1씩 off set 해준다.
            decoder_target_data[i, t-1, target_token_index[char]] = 1.

input: english <br>
output: korean 

### 2. 모델 만들기

In [76]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [77]:
batch_size = 64
epochs = 60
latent_dim = 256
num_samples = 1000

입력값 encoder_inputs는 lstm 지나면서 encoder_outputs, state_h, state_c 출력함 <br>
그럼 state_h, state_c 를 디코더에 initial state로 넣는다.

In [79]:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
encoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_8')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_8')>]

In [80]:
decoder_inputs = Input(shape = (None, num_decoder_tokens))
decoder = LSTM(latent_dim, return_sequences = True, return_state = True)

decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_outputs

<KerasTensor: shape=(None, None, 112) dtype=float32 (created by layer 'dense_4')>

In [81]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, None, 112)]  0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 256), (None, 357376      input_15[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  377856      input_16[0][0]                   
                                                                 lstm_8[0][1]              

In [82]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)


(1000, 10, 92)
(1000, 31, 112)
(1000, 31, 112)


In [83]:
model.compile(loss  = 'categorical_crossentropy', optimizer = 'adam')
# early_stop = EarlyStopping(monitor = 'var_lostt', patience = 3)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/60
13/13 [==============================] - 27s 1s/step - loss: 1.7176 - val_loss: 1.6475
Epoch 2/60
13/13 [==============================] - 11s 828ms/step - loss: 1.4022 - val_loss: 1.5100
Epoch 3/60
13/13 [==============================] - 8s 589ms/step - loss: 1.3182 - val_loss: 1.4767
Epoch 4/60
13/13 [==============================] - 8s 588ms/step - loss: 1.2948 - val_loss: 1.4727
Epoch 5/60
13/13 [==============================] - 9s 745ms/step - loss: 1.2869 - val_loss: 1.4606
Epoch 6/60
13/13 [==============================] - 9s 701ms/step - loss: 1.2774 - val_loss: 1.4523
Epoch 7/60
13/13 [==============================] - 8s 613ms/step - loss: 1.2683 - val_loss: 1.4431
Epoch 8/60
13/13 [==============================] - 8s 591ms/step - loss: 1.2627 - val_loss: 1.4389
Epoch 9/60
13/13 [==============================] - 8s 589ms/step - loss: 1.2552 - val_loss: 1.4363
Epoch 10/60
13/13 [==============================] - 8s 604ms/step - loss: 1.2504 - val_loss: 1.4351


# 추론모델 생성
1. 입력문장을 encode하고 초기 상태의 decoder을 가져온다.
2. 초기상태 decoder의 한 단계와 시퀀스 시작 토큰을 목표로 실행. 출력은 다음 목표 문자.
3. 예측된 목표 문자를 붙이고 반복

In [84]:
# encoder 추론 모델 만들기
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

# decoder 추론 모델 만들기
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states )
decoder_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, None, 92)]        0         
_________________________________________________________________
lstm_8 (LSTM)                [(None, 256), (None, 256) 357376    
Total params: 357,376
Trainable params: 357,376
Non-trainable params: 0
_________________________________________________________________
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, None, 112)]  0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 256)]        0                                            
_

In [85]:
# 숫자를 문자로 변환
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
for key, value in reverse_input_char_index.items():
    if key < 10:
        print(key, ". ", value)

0 .   
1 .  !
2 .  "
3 .  $
4 .  %
5 .  '
6 .  (
7 .  )
8 .  +
9 .  ,


In [86]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '"',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: '+',
 12: ',',
 13: '-',
 14: '.',
 15: '/',
 16: '0',
 17: '1',
 18: '2',
 19: '3',
 20: '4',
 21: '5',
 22: '6',
 23: '7',
 24: '8',
 25: '9',
 26: ':',
 27: ';',
 28: '?',
 29: 'A',
 30: 'B',
 31: 'C',
 32: 'D',
 33: 'E',
 34: 'F',
 35: 'G',
 36: 'H',
 37: 'I',
 38: 'J',
 39: 'K',
 40: 'L',
 41: 'M',
 42: 'N',
 43: 'O',
 44: 'P',
 45: 'Q',
 46: 'R',
 47: 'S',
 48: 'T',
 49: 'U',
 50: 'V',
 51: 'W',
 52: 'X',
 53: 'Y',
 54: 'Z',
 55: 'a',
 56: 'b',
 57: 'c',
 58: 'd',
 59: 'e',
 60: 'f',
 61: 'g',
 62: 'h',
 63: 'i',
 64: 'j',
 65: 'k',
 66: 'l',
 67: 'm',
 68: 'n',
 69: 'o',
 70: 'p',
 71: 'q',
 72: 'r',
 73: 's',
 74: 't',
 75: 'u',
 76: 'v',
 77: 'w',
 78: 'x',
 79: 'y',
 80: 'z',
 81: '\xa0',
 82: '¡',
 83: '¨',
 84: 'ª',
 85: '«',
 86: '°',
 87: '´',
 88: 'º',
 89: '»',
 90: '¿',
 91: 'Á',
 92: 'É',
 93: 'Í',
 94: 'Ó',
 95: 'Ú',
 96: 'á',
 97: 'é',
 98: 'ê',
 99: 'í',
 100:

In [87]:
input_seq = encoder_input_data[3: 4]
print(decode_sequence(input_seq))
print(eng_data[3])

V
a
a
.


Vaa.

Go.


In [90]:
def decode_sequence(input_seq):
    # one hot으로 만든 데이터 입력
    states_value = encoder_model.predict(input_seq)
    
    #디코더의 입력으로 쓸 값
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # 시작 문자는 \t로 시작
    target_seq[0,0, target_token_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 이전의 값 디코더에 넣어서 새로운 출력값 얻음
        # 이전 문자와 상태 데이토로 다음 문장 상태 얻음
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        #one hot을 실제 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        decoded_sentence += sampled_char

        # 종료문자 오거나 문장길이 초과되면 종료
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] = 1.
        
        states_value = [h, c]
    return(decoded_sentence)

for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('입력문장', eng_data[seq_index])
    print('출력문장', decoded_sentence)

입력문장 Go.
출력문장 Vaa.

입력문장 Go.
출력문장 Vaa.

입력문장 Go.
출력문장 Vaa.

입력문장 Go.
출력문장 Vaa.

입력문장 Hi.
출력문장 Sant

입력문장 Run!
출력문장 ¡Caraa

입력문장 Run!
출력문장 ¡Caraa

입력문장 Run!
출력문장 ¡Caraa

입력문장 Run!
출력문장 ¡Caraa

입력문장 Run.
출력문장 Solate.



In [93]:
help(random)

Help on module random:

NAME
    random - Random variable generators.

MODULE REFERENCE
    https://docs.python.org/3.8/library/random
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
        integers
        --------
               uniform within range
    
        sequences
        ---------
               pick random element
               pick random sample
               pick weighted random sample
               generate random permutation
    
        distributions on the real line:
        ------------------------------
               uniform
               triangular
               normal (Gaussian)
               lognormal
               negative exponential
               gamma
             

In [92]:
eng_data[:10]

['Go.', 'Go.', 'Go.', 'Go.', 'Hi.', 'Run!', 'Run!', 'Run!', 'Run!', 'Run.']

# 결과 잘 안만들어져서 다시 만들기
https://wikidocs.net/24996

In [66]:
src_vocab = input_characters
tar_vocab = target_characters